In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [94]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

def cluster_composition_and_centroid(self):
## cluster composition and centroids

result = dict(zip(list(self.correlation_matrix.columns), self.apply_SPONGE())) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()

## constituent_weights
centred_returns = df_cleaned.copy()
sigma = 0.01

constituent_weights = pd.DataFrame(index=['Weight'], columns=centred_returns.columns)
total_weight = pd.DataFrame(index=['Total weight'], columns=[i for i in range(number_of_clusters)], data=np.zeros((1, number_of_clusters)))

## we first compute the difference between the cluster centroid return and the cluster ticker return
for ticker in centred_returns.columns:
    centred_returns[ticker][:-1] = centred_returns[ticker][:-1] - centroid_returns[int(centred_returns[ticker]['Cluster'])]

## we use this difference to compute the distance between each asset and its cluster centroid return 
for ticker in centred_returns.columns:
    constituent_weights[ticker] = np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)
    total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight'] += np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)

## we normalize the weights
for ticker in centred_returns.columns:
    constituent_weights[ticker] = constituent_weights[ticker]['Weight']/total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight']

## check whether the weights equal to 0 within each cluster: 
# constituent_weights[[ticker for ticker in centred_returns.columns if centred_returns[ticker]['Cluster']  == 1.0]].sum(axis=1)
    
## cluster returns 
    
cluster_returns = pd.DataFrame(index=df_cleaned.index[:-1], columns=np.arange(number_of_clusters), data=np.zeros((df_cleaned.shape[0] - 1, number_of_clusters)))

for ticker in df_cleaned.columns:
    cluster_returns[int(df_cleaned[ticker]['Cluster'])] = cluster_returns[int(df_cleaned[ticker]['Cluster'])] + constituent_weights[ticker]['Weight'] * df_cleaned[ticker][:-1]

IndentationError: expected an indented block after function definition on line 6 (2352418720.py, line 9)

In [4]:
from pypfopt.efficient_frontier import EfficientFrontier
import process
from PyFolioCC import PyFolioCC 

historical_data = df_cleaned
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)

TypeError: unsupported type: <class 'numpy.float64'>

In [7]:
cluster_return.mean()

cluster 1    -0.005723
cluster 2    -0.001280
cluster 3    -0.001702
cluster 4     0.000007
cluster 5    -0.004083
cluster 6     0.002737
cluster 7    -0.000691
cluster 8    -0.000855
cluster 9    -0.004311
cluster 10    0.001726
cluster 11   -0.000147
cluster 12    0.001738
cluster 13   -0.000769
cluster 14   -0.003506
cluster 15   -0.000850
cluster 16   -0.004075
cluster 17   -0.000703
cluster 18   -0.000968
cluster 19   -0.001793
cluster 20    0.001216
cluster 21   -0.000608
cluster 22   -0.000837
cluster 23   -0.001091
cluster 24   -0.003312
cluster 25   -0.000491
cluster 26   -0.000896
cluster 27   -0.000612
cluster 28   -0.002951
cluster 29    0.000846
cluster 30   -0.002832
cluster 31   -0.002775
cluster 32    0.001943
cluster 33   -0.004813
cluster 34   -0.005190
cluster 35    0.000575
cluster 36   -0.002211
cluster 37   -0.001629
cluster 38    0.000319
dtype: float64

In [30]:
df_cleaned.transpose()['AA'][lookback_window[0]:lookback_window[1]]

03/01/2000   -0.013042
04/01/2000    0.010043
05/01/2000    0.047628
06/01/2000   -0.011713
07/01/2000   -0.016118
10/01/2000   -0.032073
11/01/2000    0.022608
12/01/2000   -0.005249
13/01/2000   -0.018210
14/01/2000   -0.020109
18/01/2000   -0.033100
19/01/2000    0.001601
20/01/2000   -0.028502
21/01/2000    0.015038
24/01/2000   -0.034299
25/01/2000   -0.015438
26/01/2000   -0.015666
27/01/2000   -0.021053
28/01/2000   -0.024868
31/01/2000   -0.002687
01/02/2000    0.019248
02/02/2000    0.039321
03/02/2000   -0.031505
04/02/2000   -0.027974
07/02/2000   -0.038691
08/02/2000   -0.045399
09/02/2000   -0.019158
10/02/2000    0.007663
11/02/2000    0.019581
14/02/2000    0.053299
15/02/2000    0.028316
16/02/2000    0.000848
17/02/2000    0.038928
18/02/2000   -0.018107
22/02/2000   -0.028889
23/02/2000   -0.026021
24/02/2000   -0.025676
25/02/2000    0.008047
28/02/2000    0.020436
29/02/2000   -0.035846
01/03/2000    0.018997
02/03/2000   -0.032216
03/03/2000    0.023268
06/03/2000 

In [31]:
cluster_returns = pd.DataFrame(index = df_cleaned.columns[lookback_window[0]:lookback_window[1]], columns= [f'cluster {i}' for i in range(1, number_of_clusters + 1)], data = np.zeros((len(df_cleaned.columns[lookback_window[0]:lookback_window[1]]), len(constituent_weights) + 1)))

for cluster in constituent_weights.keys():
    for ticker, weight in constituent_weights[cluster].items():
        cluster_returns[cluster] = cluster_returns[cluster] + df_cleaned.transpose()[ticker][lookback_window[0]:lookback_window[1]]*weight

In [60]:
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)
final_weights = process.final_weights(markowitz_weights=markowitz_weights_res, constituent_weights=constituent_weights)
print(corr_matrix.shape, len(cluster_comp), len(constituent_weights), len(cluster_return), len(markowitz_weights_res), len(final_weights))


(695, 695) 37 37 37 37 692


In [10]:
cluster_returns = pd.DataFrame(index = [f'cluster {i}' for i in range(1, len(constituent_weights) + 1)], columns = df_cleaned.columns[lookback_window[0]:lookback_window[1]])
for keys, value in constituent_weights['cluster 1'].items():
    print(keys, value)

AVA 3.921716041425027e-05
BKH 0.110883916406304
BXS 0.2035512296849513
DRQ 0.04082643926897566
INCY 1.0230500389790201e-11
LNT 0.1654632187161882
MTZ 0.14365464588183466
NI 0.05214005001251528
OKE 0.2394986472578898
PKI 0.01829458705579904
RMD 0.02564804854489745
